# Archived Pillow Method

In [1]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import arabic_reshaper
from bidi.algorithm import get_display

def create_text_image(df, arabic_column, english_column, image_path='output_image.png'):
    # Load a font
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Define text wrapping parameters
    wrapper = textwrap.TextWrapper(width=35, expand_tabs=False, replace_whitespace=False)

    # Calculate total height required for all text
    total_height = 0
    for index, row in df.iterrows():
        arabic_lines = wrapper.wrap(text=row[arabic_column])
        english_lines = wrapper.wrap(text=row[english_column])
        total_height += (len(arabic_lines) + len(english_lines)) * 25 + 10  # Line height + spacing

    # Set image dimensions
    img_width = 800
    img_height = total_height + 50  # Additional padding
    img = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(img)

    # Starting position for text
    y_position = 10

    for index, row in df.iterrows():
        # Wrap Arabic and English texts
        arabic_lines = wrapper.wrap(text=row[arabic_column])
        english_lines = wrapper.wrap(text=row[english_column])

        # Reshape and reorder Arabic text
        reshaped_arabic = arabic_reshaper.reshape(row[arabic_column])
        bidi_arabic = get_display(reshaped_arabic)

        # Now wrap the processed Arabic text instead of the original
        arabic_lines = wrapper.wrap(text=bidi_arabic)
        english_lines = wrapper.wrap(text=row[english_column])
        
        # Draw Arabic text
        for line in arabic_lines:
            draw.text((10, y_position), line, font=font, fill='black')
            y_position += 25

        # Draw English text
        for line in english_lines:
            draw.text((img_width // 2, y_position), line, font=font, fill='black')
            y_position += 25

        # Add spacing between rows
        y_position += 10

    # Save the image
    img.save(image_path)
    img.show()